In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from typing import Union, List
import joblib
import imblearn
import warnings
from collections import Counter
from imblearn.over_sampling import SMOTE

#warnings.filterwarnings(action='ignore')

## load data

In [4]:
df = pd.read_csv('modified_log_var25_0426.csv')

In [5]:
#df = p.get_df()
df.shape

(1131682, 26)

In [7]:
scaler = MinMaxScaler()

scdf = scaler.fit_transform(df)
scdf = pd.DataFrame(scdf)
scdf

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.788616,0.578659,0.0,0.0,0.328475,0.2,0.512129,0.103583,1.0,1.0,...,0.117647,0.0,0.697662,0.622291,0.691673,1.0,0.0,0.0,0.0,0.0
1,0.841506,0.626931,0.0,0.0,0.517556,1.0,0.053908,0.766355,1.0,1.0,...,0.676471,1.0,0.858478,0.746129,0.713436,1.0,0.0,0.0,0.0,0.0
2,0.629513,0.598518,0.0,0.0,0.424257,1.0,0.862534,0.068536,1.0,1.0,...,0.029412,0.0,0.742873,0.561947,0.790299,1.0,0.0,0.0,0.0,1.0
3,0.687205,0.584349,0.0,0.0,0.300849,0.5,0.215633,0.198988,1.0,1.0,...,0.205882,0.0,0.785201,0.298221,0.657291,1.0,0.0,0.0,0.0,1.0
4,0.000000,0.594195,0.0,0.0,0.492084,1.0,0.080863,0.182243,1.0,0.0,...,0.205882,0.0,0.762826,0.647718,0.669366,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131677,0.000000,0.594195,0.0,0.0,0.440878,0.2,0.188679,0.465732,1.0,0.0,...,0.411765,1.0,0.796052,0.429656,0.708296,1.0,0.0,0.0,0.0,0.0
1131678,0.000000,0.563202,0.0,0.0,0.434974,0.7,0.026954,0.571651,1.0,0.0,...,0.588235,0.0,0.721832,0.298221,0.631528,1.0,0.0,0.0,0.0,1.0
1131679,0.000000,0.603287,0.0,0.0,0.465516,0.4,0.404313,0.053349,1.0,0.0,...,0.058824,0.0,0.798741,0.601347,0.703171,1.0,0.0,0.0,0.0,1.0
1131680,0.000000,0.577445,0.0,0.0,0.380483,0.9,0.161725,0.351246,1.0,0.0,...,0.294118,0.0,0.766240,0.272123,0.600121,1.0,0.0,0.0,0.0,0.0
